In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("wine_data/winequality-white.csv", sep=";")
df

In [ ]:
input_columns = df.columns[:11]
output_column = df.columns[11]

In [ ]:
input_df = df[input_columns]
output_df = df[output_column]

In [ ]:
input_np = input_df.to_numpy(dtype=np.float32)
input_t = torch.tensor(input_np)

In [ ]:
output_np = output_df.to_numpy(dtype=np.float32)
output_t = torch.tensor(output_np).unsqueeze(1)

In [ ]:
# Training Validation Split
batch = input_t.size(0)

shuffled_indices = torch.randperm(batch)
training_indices = shuffled_indices[:int(batch * 0.8)]
validation_indices = shuffled_indices[int(batch * 0.8):]

training_input_t = input_t[training_indices]
training_output_t = output_t[training_indices]

validation_input_t = input_t[validation_indices]
validation_output_t = output_t[validation_indices]

In [ ]:
# Input Normalisation
mean_training = torch.mean(training_input_t, dim=0)
sd_training = torch.std(training_input_t, dim=0)

training_input_t = (training_input_t - mean_training) / sd_training
validation_input_t = (validation_input_t - mean_training) / sd_training

In [ ]:
seq_model = nn.Sequential(
  nn.Linear(11, 110),
  nn.Tanh(),
  nn.Linear(110, 1)
)

In [ ]:
learning_rate = 1e-2
optimizer = optim.SGD(seq_model.parameters(), learning_rate)

In [ ]:
def training_loop(epochs: int, model: nn.Sequential, optimizer: optim.SGD, loss_function: nn.MSELoss, training_input: torch.Tensor, training_output: torch.Tensor,
                  validation_input: torch.Tensor, validation_output: torch.Tensor) -> nn.Sequential:
  for epoch in range(1, epochs + 1):
    training_pred_output = model(training_input)
    training_loss = loss_function(training_pred_output, training_output)

    with torch.no_grad():
      validation_pred_output = model(validation_input)
      validation_loss = loss_function(validation_pred_output, validation_output)

    optimizer.zero_grad()
    training_loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
      print(f"Epoch: {epoch}, Training Loss: {training_loss}, Validation Loss: {validation_loss}")

  return model

In [ ]:
trained_model = training_loop(10000, seq_model, optimizer, nn.MSELoss(), training_input_t, training_output_t, validation_input_t, validation_output_t)

In [ ]:
pred_score = trained_model(validation_input_t)
pred_score